## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marsaxlokk,MT,35.8419,14.5431,81.09,78,0,11.77,clear sky
1,1,Hervey Bay,AU,-25.2986,152.8535,68.00,81,100,10.25,overcast clouds
2,2,Taoudenni,ML,22.6783,-3.9836,95.88,15,2,4.23,clear sky
3,3,Jamestown,US,42.0970,-79.2353,64.18,64,0,5.75,clear sky
4,4,Barrow,US,71.2906,-156.7887,37.42,87,75,17.27,mist


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                225
City                   225
Country                225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsaxlokk,MT,81.09,clear sky,35.8419,14.5431,
5,Maurawan,IN,80.64,overcast clouds,26.4333,80.8833,
11,San Quintin,MX,77.20,light rain,30.4833,-115.9500,
12,Thinadhoo,MV,81.16,overcast clouds,0.5333,72.9333,
13,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,
15,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,
20,Yining,CN,81.70,overcast clouds,29.0361,114.5636,
27,Karaton,ID,77.77,light rain,-6.3424,106.1045,
35,Mizdah,LY,76.10,clear sky,31.4451,12.9801,
37,Pacific Grove,US,87.39,clear sky,36.6177,-121.9166,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + ": Hotel Name in <" + hotel_df["City"] + "> not found... skipping.")
        pass
        

0      12: Hotel Name in <Marsaxlokk> not found... sk...
5      12: Hotel Name in <Maurawan> not found... skip...
11     12: Hotel Name in <San Quintin> not found... s...
12     12: Hotel Name in <Thinadhoo> not found... ski...
13      12: Hotel Name in <Kapaa> not found... skipping.
                             ...                        
692    12: Hotel Name in <Tomatlan> not found... skip...
695    12: Hotel Name in <Phonhong> not found... skip...
696    12: Hotel Name in <Pedernales> not found... sk...
698    12: Hotel Name in <Papetoai> not found... skip...
701    12: Hotel Name in <Punto Fijo> not found... sk...
Name: City, Length: 225, dtype: object
0      105: Hotel Name in <Marsaxlokk> not found... s...
5      105: Hotel Name in <Maurawan> not found... ski...
11     105: Hotel Name in <San Quintin> not found... ...
12     105: Hotel Name in <Thinadhoo> not found... sk...
13     105: Hotel Name in <Kapaa> not found... skipping.
                             ...                 

In [9]:
# 7. Drop the rows where there is no Hotel Name

clean_hotel_df = hotel_df[(hotel_df["Hotel Name"] != "")]

# Only keep the first 180 rows
clean_hotel_df = clean_hotel_df.head(180)

# Confirm the data looks good
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Marsaxlokk,MT,81.09,clear sky,35.8419,14.5431,Water's Edge
5,Maurawan,IN,80.64,overcast clouds,26.4333,80.8833,Mahi Sanstha
11,San Quintin,MX,77.20,light rain,30.4833,-115.9500,Don Eddie's Sport Fishing Center
13,Kapaa,US,84.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
20,Yining,CN,81.70,overcast clouds,29.0361,114.5636,Xiushui Hotel
27,Karaton,ID,77.77,light rain,-6.3424,106.1045,Parkir Mobil Wisata CAS Water Park
35,Mizdah,LY,76.10,clear sky,31.4451,12.9801,محمدالمبروك الاريقط
37,Pacific Grove,US,87.39,clear sky,36.6177,-121.9166,Lovers Point Inn
45,Alofi,NU,82.29,broken clouds,-19.0595,-169.9187,Taloa Heights


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [25]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, -35.0), zoom_level=2.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))